In [1]:
import datetime
import os
import tempfile
import shutil
import time

# Main entry point for DataFrame and SQL functionality.
from pyspark.sql import SparkSession
# Start SPARK Session
spark = SparkSession.builder.appName('Basics').getOrCreate()

from pyspark.sql.functions import *
from pyspark.sql.types import *

path = "/home/bluepi/Downloads/Update/Updated Product/Latest Product/"

In [2]:
data_schema = [
        StructField('p_id', IntegerType(), True),
        StructField('p_name', StringType(), True),
        StructField('price', IntegerType(), True),
        StructField('date_timestamp', TimestampType(), True),
        StructField('record_type', StringType(), True)
    ]
    # Schema which we are accepting
final_struc = StructType(fields=data_schema)
    

In [3]:
# mainTable = spark.read.csv(
#             "/home/bluepi/Downloads/Update/product_info/main table/main_table_new.csv", 
#     header=True, schema=final_struc)

mainTable = spark.read.format('csv').options(header=True, schema=final_struc).load("/home/bluepi/Downloads/Update/product_info/main table/main_table_new.csv")
mainTable = mainTable.withColumn('record_type', lit("A"))

for i in range(10, 0, -1):
    
#     mainTable = spark.createDataFrame([], schema)

    if i != 10:
        mainTable = spark.read.format('csv').options(header=True, schema=final_struc).load(path)
        
    
 # ###################################################################################################
   
    # Address to the product_info folder
    address = "/home/bluepi/Downloads/Update/product_info/"
    previous_day = (datetime.datetime.today() -
                    datetime.timedelta(days=i)).strftime('%d-%m-%Y')
    print("\n\t\ Date ----> "+previous_day+"\n")

    # Address to the Previous Day folder
    new_address = address + previous_day
    
    per_day_data = spark.read.format('csv') \
        .options(header=True, inferschema=True) \
        .load(new_address)
################################################################

# Code Section

################################################################
    
    print(per_day_data.count())
    
#     per_day_data = per_day_data.unionByName(mainTable)
#     print(per_day_data.count())
    
    mainTable = mainTable.union(per_day_data)
    print(mainTable.count())

#     spark.catalog.refreshByPath(path)
    print("union successful")

    mainTable.coalesce(1).write.csv(path, mode= 'overwrite', header=True)
    print("file written")
#     spark.catalog.clearCache()
#     spark.catalog.refreshByPath(path)
#     print("cache cleared")
    
    
    


	\ Date ----> 05-04-2020

50
150
union successful
file written

	\ Date ----> 06-04-2020

29
179
union successful


Py4JJavaError: An error occurred while calling o67.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scal

In [4]:
import tempfile
path = "/home/bluepi/Downloads/Update/Updated Product/Latest Product/"
f  =  tempfile.TemporaryDirectory(dir=path)
f.name

'/home/bluepi/Downloads/Update/Updated Product/Latest Product/tmp6cv13bl1'